<a href="https://colab.research.google.com/github/sngo/llms-practice/blob/main/taskmanagement/TaskManagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q gradio>=4.0.0

In [0]:
import gradio as gr
import openai
import json
import html
from openai import OpenAI
import random
import datetime
from google.colab import userdata

In [0]:
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)
model = "gpt-4.1-mini"

In [0]:
def abcd_taskTool():
  tasks = [
         {"taskId": "T001", "accNo": "1234567890", "status": "Pending", "description": "Update account details"},
        {"taskId": "T002", "accNo": "9876543210", "status": "In Progress", "description": "Verify transaction"}
    ]

  probability_of_tasks = 0.8
  if random.random() < probability_of_tasks:
    return tasks
  else:
    return []

In [0]:
def abcd_NotifyTool():
    # Mock notification function
    return True

In [0]:
# Tool definitions for OpenAI
tools = [
    {
        "type": "function",
        "function": {
            "name": "abcd_taskTool",
            "description": "Retrieve a list of ABCD tasks.",
            "parameters": {}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "abcd_NotifyTool",
            "description": "Notify the support team about tasks.",
            "parameters": {}
        }
    }
]

In [0]:
# Function to mask first four digits of accNo
def mask_accNo(accNo):
    return "****" + accNo[4:] if len(accNo) >= 4 else accNo

In [0]:
# Function to format tasks as an HTML table
def format_tasks_as_table(tasks):
    if not tasks:
        return "No tasks found."

    table = "<table border='1' style='border-collapse: collapse; width: 100%;'>"
    table += "<tr><th>Task ID</th><th>Account Number</th><th>Status</th><th>Description</th></tr>"
    for task in tasks:
        table += "<tr>"
        table += f"<td>{html.escape(task.get('taskId', ''))}</td>"
        table += f"<td>{html.escape(mask_accNo(task.get('accNo', '')))}</td>"
        table += f"<td>{html.escape(task.get('status', ''))}</td>"
        table += f"<td>{html.escape(task.get('description', ''))}</td>"
        table += "</tr>"
    table += "</table>"
    return table

In [0]:
# Chat function to handle user input and bot responses with tool calling
def chat_function(message, history):
    # Append user message to history
    print(f"User message: {message}")

    history.append([message, None])
    print(f"History: {history}")

    # Prepare messages for OpenAI API
    messages = [{"role": "system", "content": (
        "You are a helpful assistant. When the user asks about checking ABCD tasks, call the abcd_taskTool function. "
        "If tasks are returned, ask the user if they want to notify the support team. "
        "If the user agrees to notify, call the abcd_NotifyTool function. "
        "If no tasks are found, respond with a cheerful message. "
        "If the user declines notification, respond creatively."
    )}]
    for user_msg, bot_msg in history[:-1]:
        if user_msg:
            messages.append({"role": "user", "content": user_msg})
        if bot_msg:
            messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": message})

    # Handle state for notification response
    state = history[-2][1] if len(history) > 1 else None
    if state and "Would you like to notify the support team?" in state:
        if message.lower() in ["yes", "y", "yeah", "sure", "ok", "notify", "yep"]:
            # Simulate tool call for notification
            notify_result = abcd_NotifyTool()
            response = "The support team has been notified successfully! They'll take it from here. 😊"
            history[-1][1] = response
            yield history, "normal", ""
            return
        else:
            response = (
                "Alright, we'll hold off on notifying the team. 🌟 "
                "How about we explore something fun, like planning your next big adventure?"
            )
            history[-1][1] = response
            yield history, "normal", ""
            return

    try:
        # Call OpenAI API with tool calling
        response = client.chat.completions.create(
            model= model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_tokens=300
        )

        # Process response
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls

        if tool_calls:
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                if function_name == "abcd_taskTool":
                    tasks = abcd_taskTool()
                    task_count = len(tasks)

                    if task_count > 0:
                        table = format_tasks_as_table(tasks)
                        response = (
                            f"There are {task_count} tasks found:<br>{table}<br>"
                            "Would you like to notify the support team? (Yes/No)"
                        )
                        history[-1][1] = response
                        print(f"History after tool call: {history}")
                        print(f"History at -1, 1: {history[-1][1]}")
                        yield history, "waiting_for_notify_response", ""
                    else:
                        response = (
                            "Hooray! No ABCD tasks to worry about! 🎈 "
                            "You're free as a bird—any fun plans for the day?"
                        )
                        history[-1][1] = response
                        yield history, "normal", ""
                elif function_name == "abcd_NotifyTool":
                    abcd_NotifyTool()
                    response = "The support team has been notified successfully! They'll take it from here. 😊"
                    history[-1][1] = response
                    yield history, "normal", ""
        else:
            # No tool calls, use the LLM's response
            bot_response = response_message.content
            history[-1][1] = bot_response
            yield history, "normal", ""

    except Exception as e:
        history[-1][1] = f"Error: {str(e)}"
        yield history, "normal", ""

In [0]:
# Custom CSS for table and chat styling
custom_css = """
table {
    font-family: Arial, sans-serif;
    margin: 10px 0;
}
th, td {
    padding: 8px;
    text-align: left;
}
th {
    background-color: #f2f2f2;
}
tr:nth-child(even) {
    background-color: #f9f9f9;
}
"""

In [0]:
# Create Gradio interface
with gr.Blocks(css=custom_css) as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type your message here...")
    state = gr.State(value="normal")

    msg.submit(
        chat_function,
        inputs=[msg, chatbot],
        outputs=[chatbot, state, msg]
    )

In [0]:
demo.launch(debug=True, share=True)